#### Book Recommendation System using Machine Learning

using Collabarative filtering-based Recommendation System

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [ ]:

books = pd.read_csv("data/BX-Books.csv", sep=";", encoding="latin-1", on_bad_lines='skip')
user = pd.read_csv("data/BX-Users.csv", sep=";", encoding="latin-1", on_bad_lines='skip')
ratings = pd.read_csv("data/BX-Book-Ratings.csv", sep=";", encoding="latin-1", on_bad_lines='skip')

C:\Users\Ashwanth\AppData\Local\Temp\ipykernel_19180\984543385.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv("data/BX-Books.csv", sep=";", encoding="latin-1", on_bad_lines='skip')


In [3]:
books = books[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',  'Image-URL-L']]
books.rename(columns={
    'Book-Title': 'title',
    'Book-Author': 'author',
    'Year-Of-Publication': 'year',
    'Publisher': 'publisher',
    'Image-URL-L': 'image_url'
}, inplace=True)

user.rename(columns={
    'User-ID': 'user_id',
    'Location': 'location',
    'Age': 'age'
}, inplace=True)

ratings.rename(columns={
    'User-ID': 'user_id',
    'Book-Rating': 'rating'
}, inplace=True)

In [4]:
X = ratings['user_id'].value_counts() > 200
y = X[X].index
ratings = ratings[ratings['user_id'].isin(y)]

ratings_books = ratings.merge(books, on='ISBN', how='left')

num_ratings = ratings_books.groupby('title')['rating'].count().reset_index()
num_ratings.rename(columns={'rating': 'num_of_ratings'}, inplace=True)

final_ratings = ratings_books.merge(num_ratings, on='title')
final_ratings = final_ratings[final_ratings['num_of_ratings'] >= 50]
final_ratings.shape

(61853, 9)

In [5]:
final_ratings.drop_duplicates(['user_id', 'title'], inplace=True)

book_pivot = final_ratings.pivot_table(index='title', columns='user_id', values='rating')
book_pivot.fillna(0, inplace=True)

book_sparse = csr_matrix(book_pivot.values) 

In [6]:
model = NearestNeighbors(algorithm='brute')
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [7]:
distance, suggestions = model.kneighbors(book_pivot.iloc[277,:].values.reshape(1,-1), n_neighbors=6)

In [8]:
book_names = book_pivot.index